In [24]:
import pandas as pd
jeopardy=pd.read_csv('jeopardy.csv')
jeopardy.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [3]:
jeopardy.columns

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')

In [25]:
jeopardy.columns=['Show Number', 'Air Date', 'Round', 'Category', 'Value',
       'Question', 'Answer']

In [26]:
import re
def normalise(string):
    string=string.lower()
    string = re.sub("[^A-Za-z0-9\s]", "", string)
    return string

jeopardy['clean_question']=jeopardy['Question'].apply(normalise)
jeopardy['clean_answer']=jeopardy['Answer'].apply(normalise)


    

In [6]:
jeopardy['clean_question']

0        for the last 8 years of his life galileo was u...
1        no 2 1912 olympian football star at carlisle i...
2        the city of yuma in this state has a record av...
3        in 1963 live on the art linkletter show this c...
4        signer of the dec of indep framer of the const...
5        in the title of an aesop fable this insect sha...
6        built in 312 bc to link rome  the south of ita...
7        no 8 30 steals for the birmingham barons 2306 ...
8        in the winter of 197172 a record 1122 inches o...
9        this housewares store was named for the packag...
10                                          and away we go
11       cows regurgitate this from the first stomach t...
12       in 1000 rajaraja i of the cholas battled to ta...
13       no 1 lettered in hoops football  lacrosse at s...
14       on june 28 1994 the natl weather service began...
15       this companys accutron watch introduced in 196...
16       outlaw murdered by a traitor and a coward whos.

In [27]:
def normalise_int(val):
    val = re.sub("[^A-Za-z0-9\s]", "", val)
    try:
        val=int(val)
    except Exception:
        val=0
    return val

jeopardy['clean_value']=jeopardy['Value'].apply(normalise_int)

In [8]:
jeopardy['clean_value']

0         200
1         200
2         200
3         200
4         200
5         200
6         400
7         400
8         400
9         400
10        400
11        400
12        600
13        600
14        600
15        600
16        600
17        600
18        800
19        800
20        800
21        800
22       2000
23        800
24       1000
25       1000
26       1000
27       1000
28       1000
29        400
         ... 
19969    1200
19970    1200
19971    1500
19972    1200
19973    1200
19974    1200
19975    1600
19976    1600
19977    1600
19978    1600
19979    1600
19980    1600
19981    1200
19982    2000
19983    2000
19984    2000
19985    2000
19986    2000
19987       0
19988     100
19989     100
19990     100
19991     100
19992     100
19993     100
19994     200
19995     200
19996     200
19997     200
19998     200
Name: clean_value, Length: 19999, dtype: int64

In [28]:
jeopardy["Air Date"] = pd.to_datetime(jeopardy["Air Date"])


In [29]:
def answerin_question(row):
    split_answer=row['clean_answer'].split(' ')
    split_question = row["clean_question"].split(" ")

    match_count=0
    if 'the' in split_answer:
        split_answer.remove('the')
    if len(split_answer)==0:
        return 0
    for each in split_answer:
        if each in split_question:
            match_count+=1
    return match_count/len(split_answer)

jeopardy['answer_in_question']=jeopardy.apply(answerin_question,axis=1)

In [14]:
jeopardy['answer_in_question'].mean()

0.06049325706933587

Answer appears in question only 6% of the times.

In [30]:
question_overlap=[]
terms_used=set()
for i, row in jeopardy.iterrows():
    split_question=row["clean_question"].split(" ")
    split_question = [q for q in split_question if len(q) > 5]
    match_count=0
    for each in split_question:
        if each in terms_used:
            match_count+=1
    for each in split_question:
        terms_used.add(each)
    if len(split_question)>0:
        match_count=match_count/len(split_question)
    question_overlap.append(match_count)
    
jeopardy['question_overlap']=question_overlap
jeopardy["question_overlap"].mean()


0.6908737315671962

There is 69% overlap between old questions an new ones.

In [31]:
def val_count(row):
    value=0
    if row['clean_value']>800:
        value=1
    
    return value
jeopardy['high_value']=jeopardy.apply(val_count,axis=1)



In [32]:
def low_high(word):
    low_count=0
    high_count=0
    for i, row in jeopardy.iterrows():
        q=row['clean_question'].split(" ")
        if word in q:
            if row["high_value"]==1:
                high_count+=1
            else:
                low_count+=1
    return high_count,low_count

observed_expected=[]
comparison_terms=list(terms_used)[:5]

    

In [33]:
for each in comparison_terms:
    observed_expected.append(low_high(each))

In [34]:
observed_expected

[(0, 1), (2, 7), (0, 1), (0, 1), (0, 2)]

In [37]:
high_value_count=jeopardy[jeopardy['high_value']==1].shape[0]
high_value_count

5734

In [40]:
low_value_count=jeopardy[jeopardy['high_value']==0].shape[0]

In [44]:
from scipy.stats import chisquare
import numpy as np
chi_squared=[]
for each in observed_expected:
    total=sum(each)
    total_prop=total/jeopardy.shape[0]
    exp_high=total_prop*high_value_count
    exp_low=total_prop*low_value_count
    observed = np.array([each[0], each[1]])
    expected = np.array([exp_high, low_value_exp])
    chi_squared.append(chisquare(observed, expected))
    

NameError: name 'high_value_exp' is not defined